In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_excel("230807_input.xlsx")
df.head()

,year,month,day,weekday,time,q_bld_load,oadb,rh,oawb,mode1,...,temp_chws2,temp_out_br_ch2,mode3,q_load3,temp_chws3,temp_out_br_ch3,mode4,q_load4,temp_chws4,temp_out_br_ch4
0,NaN,NaN,NaN,NaN,23:00:00,NaN,28.5,84.0,NaN,-1.0,...,NaN,-4.5,-1.0,NaN,NaN,-5.5,0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,23:15:00,NaN,28.3,84.2,NaN,-1.0,...,NaN,-4.5,-1.0,NaN,NaN,-5.5,0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,23:30:00,NaN,28.2,84.4,NaN,-1.0,...,NaN,-4.5,-1.0,NaN,NaN,-5.5,0,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,23:45:00,NaN,28.1,84.6,NaN,-1.0,...,NaN,-4.5,-1.0,NaN,NaN,-5.5,0,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,1900-01-01 00:00:00,NaN,28.0,84.8,NaN,-1.0,...,NaN,-4.5,-1.0,NaN,NaN,-5.5,0,NaN,NaN,NaN


In [9]:
df['mode1'][0]
lst.append(t_oawb) # [3, 3, 2 ... => 96개
df['new_lst'] = lst

-1.0

In [10]:
import math

In [ ]:
# 열교환기
def hx(train, flow_br, q_load):
    flow_chw = [0, 14018, 14018, 14018, 14018]
    ka_hx_ref = [0, 2700, 2250, 2400, 2400]
    kw_chw_pump = [0, 122.2, 129.2, 125.7, 125.7]
    
    q_hx = q_load
    temp_chwr[train] = temp_chws + (q_hx * 3024) / (flow_chw * 60)
    ka_hx[train] = ka_hx_ref[train] * (flow_br/21000) ** (1/3)
    lmtd_hx = (q_hx * 3024) / (ka_hx * 860)
    
    dtemp_br_hx = (q_hx * 3024) / (1.038 x 0.8975 x flow_br * 60)
    temp_in_br_hx = (temp_chws - (t_chwr - dtemp_br_hx) x math.exp((temp_chws - temp_chwr + dtemp_br_hx) / lmtd)) / (1 - math.exp((temp_chws - t_chwr + dtemp_br_hx) / lmtd))
    temp_out_br_hx = temp_in_br_hx + dtemp_br_hx
    
    return temp_in_br_hx, temp_out_br_hx, temp_chwr, kw_chw_pump

In [11]:
# 냉동기, 냉각탑
def chiller(train, temp_in_br_ch, temp_out_br_ch, flow_br):
    flow_cw = [0, 18900, 19000, 18800, 19000]
    q_ch_rated = [0, 1118.2, 1118.2, 1118.2, 1118.2]
    kw_ch_rated = [0, 952.6, 952.6, 952.6, 952.6]
    kw_cw_pump = [0, 156.9, 161.4, 154.7, 154.7]
    kw_ct_fan = [0, 58.8, 47.8, 61.6, 61.6]
    
    a = [0, 1.458645293, 0.040009978, -1.36368*(10**(-6)), -0.02013792, 2.06332*(10**(-6)), 2.26211*(10**(-7))]
    b = [0, 1.062254257, 0.037469286, 0.001538748, -0.040480028, 0.001323365, -0.002722094]
    c = [0, 0.4, 0.3, 0.3]
    d = [0, 3.51525454, -0.083236649, 0.000265981, 3.966633299, -0.152643525, 0.002041101, -0.217416145, 0.013288222, -0.000281392]
    
    for temp_cwr in range(oawb+10, oawb+3, -0.02):
        cap_ft = a[1] + a[2] * temp_out_br_ch + a[3] * temp_out_br_ch**2 + a[4] * temp_cwr + a[5] * temp_cwr**2 + a[6] * temp_out_br_ch * temp_cwr
        q_ch_available = cap_ft * q_ch_rated[train] * 3024
        q_ch_current = flow_br * 60 * 1.038 * 0.8975 * (temp_in_br_ch - temp_out_br_ch)
        plr = q_ch_current / q_ch_available
        
        if plr > 1:
            plr = 1
            q_ch_current = q_ch_available
            temp_out_br_ch = temp_in_br_ch - q_ch_current / (flow_br * 60 * 1.038 * 0.8975)
            # continue
        
        eir_ft = b[1] + b[2] * (temp_out_br_ch) + b[3] * temp_out_br_ch**2 + b[4] * temp_cwr + b[5] * temp_cwr**2 + b[6] * temp_out_br_ch * temp_cwr
        kw_ch_available = kw_ch_rated[train] * eir_ft * cap_ft
        eir_fplr = c[1] + c[2] * plr + c[3] * plr**2
        kw_ch_current = kw_ch_available * eir_fplr
        q_cw_current = q_ch_current + kw_ch_current * 860
        
        temp_range = q_cw_current / (flow_cw * 60)
        temp_approach = d[1] + d[2]*oawb + d[3]*oawb**2 + d[4]*temp_range + d[5]*temp_range*oawb + d[6]*oawb**2*temp_range + d[7]*temp_range**2 + d[8]*oawb*temp_range**2 + d[9]*oawb**2*temp_range**2
        temp_cws = oawb + temp_approach
        
        temp_cwr_calc = oawb + temp_approach + temp_range
        
        if abs(temp_cwr_calc - temp_cwr) < 0.01:
            break
    temp_cwr = temp_cwr_calc
    
    return plr, temp_out_br_ch, temp_in_br_ch, q_ch_current, q_ch_available, kw_ch_current, q_cw_current, temp_cws, temp_cwr

# 축열조 제빙계산
def charge(iceinventory, flow_br, temp_in_br_icestor, temp_water_icestor) :
    
    a = [0, -0.00278, -0.167946, 66.765285]
    rth_icestor_install = [0, 8500, 7000, 7875, 7000]
    fouling_loss = [0, 50, 45, 66, 60]
    rth_icestor_usable = rth_icestor_install * (1- fouling_loss[train] / 100)
    
    # 축열조 제빙완료이면 계산을 하지 않고 "운전 정지"한다
    if charge_end_icestor = True, # 운전 정지이므로, 이전 값 참조
    
    flow_br_thru_icestor = flow_br[train] * open_mixvalve_icestor / 100
    # 축열조 출구 브라인온도(T_out_BR_IceStor#)를 가정하여 반복계산한다.
    for temp_out_br_iceStor in range(temp_water_iceStor-1, temp_water_iceStor-5, -0.01)    
        ka_charge_iceStor = (a[1] * iceinventory**2 + a[2] * iceinventory + a[3]) / 300 * rth_iceStor_usable * (flow_br_thru_icestor / rth_icestor_usable / 1.8)**(1/4)
        lmtd_icestor = (temp_out_br_iceStor - temp_in_br_icestor) / math.ln(temp_water_icestor - temp_in_br_icestor) / (temp_water_icestor - temp_out_br_icestor)
        q_charge_icestor = ka_charge_icestor x lmtd_icestor * 860
        temp_out_br_icestor_calc = temp_in_br_icestor + q_charge_icestor / (flow_br_thru_icestor * 60 * 1.038 * 0.8975)
    if abs(temp_out_br_icestor_calc - temp_out_br_icestor) < 0.01, temp_out_br_icestor = temp_out_br_icestor - 0.01
    break
    temp_out_br_icestor = temp_out_br_icestor_calc

    # 이제 T_out_BR_IceStor# 가 정해졌으므로
    q_charge_icestor = flow_br_thru_icestor * 60 * 1.038 * 0.8975 * (temp_out_br_icestor - temp_in_br_icestor)
    # 축열조 출구온도가 -0.5'C 이하로 내려가면 잠열축열이 시작되는 것으로 하고, 누적(잠열)축열량(RTh)과 (잠열)축열율%를 계산한다.
    if temp_out_br_icestor <= -0.5, rth_charge_latent_icestor += (q_charge_icestor / 3024)
    iceinventory = rth_ charge_latent_icestor / rth_icestor_usable * 100
    else, rth_charge_sens_icestor += (q_charge_icestor / 3024)
    # 축열조 수온을 계산한다.
    temp_water_icsStor = temp_water_icestor - q_charge_icestor / (rth_icestor_usable x 42.211 x 60)
    # 축열조 수온이 0'C 이하로 내려가면 얼음이 어는 것이므로 계속 0'C로 해야 한다.
    if temp_water_icestor <= 0, temp_water_icetor = 0

    if iceinventory > 90 and plr < 0.2 and (temp_out_br_icestor - temp_in_br_icestor) < 0.5, charge_end_icestor = True

    return temp_out_br_icestor, temp_br_mix_icestor, flow_br_thru_icestor, q_charge_icestor, rth_charge_latent_icestor, rth_charge_sens_icestor, iceinventory, temp_water_icestor
      

# 축열조 방열계산
def discharge(iceinventory, discharge_percent_icestor, flow_br, temp_in_br_icestor, temp_br_mix_icestor, temp_water_icestor) :
    
    kw_ringblower = [0, 7.5, 7.5, 7.5, 7.5]
    
    b = [0, -0.000323, -0.187068, 41.358967]
    c = [0, -0.020222, 	2.512551, -26.288677]

    rth_icestor_install = [0, 8500, 7000, 7875, 7000]
    fouling_loss = [0, 50, 45, 66, 60]
    rth_icestor_usable = rth_icestor_install * (1- fouling_loss[train] / 100)
    
    # 축열조 빙열완료이면 계산을 하지 않고 "운전 정지"한다
    if discharge_end_icestor = True, # 운전 정지이므로, 이전 값 참조
    
    # temp_br_mix_icestor = temp_in_br_hx라고 가정한다.
    flow_br_thru_icestor = flow_br[train] * open_mixvalve_icestor / 100
    
    # Q_discharge_IceStor# (축열조 방열요구량, kcal/h)을 계산한다. 
    q_discharge_icestor = flow_br * 60 * 1.038 * 0.8975 * (temp_br_mix_icestor - temp_in_br_icestor) 
    # 축열조 통과 브라인 유량을 계산한다. 
    flow_br_thru_icestor = flow_br * open_icestor_mixvalve / 100
    flow_br_bypass_icestor = flow_br - flow_br_thru_icestor 
    # AirAgi# 상태를 판단한다.
    if discharge_percent_icestor > 50, airagi = "ON"
    else, airagi = "OFF"
    ka_discharge_noair_iceStor = (b[1] * discharge_percent_icestor**2 + b[2] * discharge_percent_icestor + b[3]) / 300 * rth_iceStor_usable * (flow_br_thru_icestor / rth_icestor_usable / 1.8)**(1/4)
    ka_discharge_airagi_iceStor = (c[1] * discharge_percent_icestor**2 + c[2] * discharge_percent_icestor + c[3]) / 300 * rth_iceStor_usable * (flow_br_thru_icestor / rth_icestor_usable / 1.8)**(1/4)
    if airagi = 'ON', ka_discharge_icestor = ka_discharge_airagi_icestor
    if airagi = 'OFF', ka_discharge_icestor = ka_discharge_noair_icestor
    lmtd_icestor = q_discharge_icestor / (ka_discharge_icestor * 860)
    dtemp_br_icestor = q_discharge_icestor / (flow_br_thru_icestor * 60 * 1.038 * 0.8975)
    temp_out_br_icestor = temp_water_icestor + (temp_in_br_icestor - temp_water_icestor) / math.exp(dtemp_br_icestor / lmtd_icestor)

    # 축열조 혼합밸브후 브라인 온도가 맞는지 검토한다.
    temp_br_mix_icestor_calc = (temp_in_br_icestor * flow_br_bypass_icestor + temp_out_br_icestor * flow_br_thru_icestor) / flow_br[train]
    if abs(temp_br_mix_icestor_calc - temp_br_mix_icestor) < 0.01
    break
    temp_br_mix_icestor = temp_br_mix_icestor_calc
    
    # temp_br_mix_icestor (축열조 혼합밸브후 브라인온도) = temp_in_br_hx (열교환기 입구 브라인온도) 라고 가정했기 때문에 
    # temp_br_mix_icestor 가 새로 계산되었기 때문에 flow_br도 달라졌기 때문에 "열교환기 계산"으로 가서 다시 계산한다.

    # 누적방열량(RTh)를 구한다.
    rth_discharge_icestor += (q_discharge_icestor /3024 / 60)
    # 방열율(%)를 구한다.
    discharge_percent_icestor = rth_discharge_icestor / rth_icestor_usable x 100
    # 축열율(%)을 구한다
    iceinventory = 100 - discharge_percent_icestor 
    # 축열조 수온을 구한다. 
    if discharge_percent_icestor <= 100, temp_water_icestor = 0
    else temp_water_icestor = temp_water_icestor - q_discharge_icestor / (rth_icestor_usable * 42.211 * 60)
    # 링블로워 소비전력을 계산한다. 
    if airagi = "OFF", kw_ringblower = 0
    kwh_ringblower += kw_ringblower / 60 
    

    # 방열 종료 판단은???
    if iceinventory < 5 and temp_out_br_icestor > (temp_chws - 1) 이면, discharge_end_icestor = True 

    return temp_out_br_icestor, temp_br_mix_icestor, flow_br_thru_icestor, 
           q_discharge_icestor, rth_discharge_icestor, discharge_percent_icestor, iceinventory, 
           temp_water_icestor, kwh_ringblower
        

In [2]:
def br_flow(open_icestor_mixvalve):
    # [0, 0, 0, 0, 0]
    flow_br = [0] * 5 
    flow_br_open0 = [0, 21858, 21000, 20373, 21000]
    flow_br_open100 = [0, 12166, 13206, 7500, 13000]
    flow_br[train] = flow_br_open0[train] - (flow_br_open0[train] - flow_br_open100[train]) * (open_icestor_mixvalve/100)
    return flow_br
    
def kw_br_pump(open_icestor_mixvalve):
    kw_br_pump = [0] * 5
    kw_br_pump_open0 = [0, 213.5, 199.5, 210.9, 210.9]
    kw_br_pump_open100 = [0, 195.0, 186.8, 175.3, 175.3]
    kw_br_pump[train] = kw_br_pump_open0[train] - (kw_br_pump_open0[train] - kw_br_pump_open100[train]) * (open_icestor_mixvalve / 100)
    return kw_br_pump

for time in range(1, 97):
    # 기온(T_OADB)과 습도(%RH)를 읽어 와서 습공기 계산공식으로부터 외기습구온도(T_OAWB)를 구한다
    oawb = oadb * arctan(0.151977 * (rh + 8.313659)**(1/2)) + arctan(oadb + rh) - arctan(rh - 1.676331) + 0.00391838 * (rh)**(3/2) * arctan(0.023101 * rh) - 4.686035
    # 전체 냉방부하(시간 09:00~22:45, q_bld_load > 0일 때)를 각 Train#(q_load[train])로 나누는 방법은 ?
    if (41 <= time <= 96) and q_bld_load > 0, 
    no_train_operation = count(value(mode[train]) > 0)
    q_load[train] = q_bld_load / (no_train_operation)
    else q_load[train] = 0
    
    for train in range(1, 5):
        firstCharge = True
        if mode == -1: # 제빙 운전
            open_icestor_mixvalve = 100
            flow_br[train] = br_flow(open_icestor_mixvalve)
            kw_br_pump[train] = kw_br_pump(open_icestor_mixvalve)
            kwh_br_pump[train] = 0
            
            for t_min in range(1, 16):
                
                #냉동기와 냉각탑계산을 실행
                if firstCharge == True:
                    temp_in_br_ch = temp_water_icestor
                    firstCharge = False
                else:
                    temp_in_br_ch = temp_br_mix_icestor
                temp_out_br_ch = df['temp_out_br_ch'+str(train)][time]
                plr, temp_out_br_ch, temp_in_br_ch, q_ch_current, q_ch_available, kw_ch_current, q_cw_current, temp_cws, temp_cwr = 
                chiller(train, temp_in_br_ch, temp_out_br_ch, flow_br[train])
                
                # "축열조 제빙계산"으로 가서 계산
                # iceinventory 초기값을 0으로 하고 누적계산한다. 
                # flow_br 앞에서 계산하여 알고 있다.  
                # temp_water_icestor 초기값을 입력 받는다. 
                # temp_in_br_icestor, 
                #        if firstChartge == True, temp_in_br_iceStor = temp_water_iceStor - 5.0 (제빙운전 처음일 때)
                #        else, temp_in_br_iceStor = temp_out_br_ch (냉동기 계산에서 구하였다.)
                temp_out_br_icestor, temp_br_mix_icestor, flow_br_thru_icestor, q_charge_icestor, rth_charge_latent_icestor, 
                rth_charge_sens_icestor, iceinventory, temp_water_icestor = 
                charge(iceinventory, flow_br, temp_in_br_icestor, temp_water_icestor)
                
                # temp_br_mix_icestor_new = temp_out_br_icestor
                # temp_br_mix_icestor_new가 temp_br_mix_icestor와 다르면 새로운 temp_br_mix_icestor로 냉동기 계산으로 간다.
                
                kwh_br_pump[train] += kw_br_pump[train] / 60
                kwh_cw_pump[train] += kw_cw_pump[train] / 60
                kwh_ctfan[train] += kw_ctfan[train] / 60
                kwh_ch_current[train] += kw_ch_current[train] / 60
                rth_ch_current[train] += q_ch_current[train] / 3024 / 60
                rth_cw_current[train] += q_cw_current[train] / 3024 / 60
                
            # 계산이 모두 끝나면, 계산과정에서 구한 모든 변수값을 계산한 timestep과 함께 저장
            
        elif mode == 2: # 병렬 운전
            kwh_br_pump = [0] * 5
            kwh_chw_pump = [0] * 5
            for t_min in range(1, 16):
                open_icestor_mixvalve = 100
                flow_br = br_flow(open_icestor_mixvalve)
                kw_br_pump = kw_br_pump(open_icestor_mixvalve)
                                
                temp_in_br_hx, temp_out_br_hx, temp_chwr, kw_chw_pump = hx(train, flow_br, q_load)
                
                # 축열조 방열 계산
                temp_out_br_icestor, temp_br_mix_icestor, flow_br_thru_icestor, q_discharge_icestor, rth_discharge_icestor, discharge_percent_icestor, iceinventory,
                temp_water_icestor, kwh_ringblower = discharge(iceinventory, discharge_percent_icestor, flow_br, temp_in_br_icestor, temp_br_mix_icestor, temp_water_icestor) :
                                
                # 냉동기와 냉각탑 계산
                    temp_in_br_ch = temp_br_out_hx
                temp_out_br_ch = df['temp_out_br_ch'+str(train)][time]
                plr, temp_out_br_ch, temp_in_br_ch, q_ch_current, q_ch_available, kw_ch_current, q_cw_current, temp_cws, temp_cwr = 
                chiller(train, temp_in_br_ch, temp_out_br_ch, flow_br[train])
                
                # temp_out_br_ch가 temp_in_br_icestor와 깉지 않으면 open_icestor_mixvalve를 감소시켜 다시 계산
                
                kwh_br_pump[train] += kw_br_pump / 60
                kwh_chw_pump[train] += kw_chw_pump / 60
                rth_hx[train] += q_hx[train] / 3024 / 60
                rth_discharge_icestor[train] += q_discharge_icestor[train] / 3024 / 60
                kwh_cw_pump[train] += kw_cw_pump[train] / 60
                kwh_ctfan[train] += kw_ctfan[train] / 60
                kwh_ch_current[train] += kw_ch_current[train] / 60
                rth_ch_current[train] += q_ch_current[train] / 3024 / 60
                rth_cw_current[train] += q_cw_current[train] / 3024 / 60
                        
                
        elif mode == 1.2: # 냉동기 단독 운전
            kwh_br_pump = [0] * 5
            kwh_chw_pump = [0] * 5
            for t_min in range(1, 16):
                open_icestor_mixvalve = 0
                flow_br = br_flow(open_icestor_mixvalve)
                kw_br_pump = kw_br_pump(open_icestor_mixvalve)
                                
                temp_in_br_hx, temp_out_br_hx, temp_chwr, kw_chw_pump = hx(train, flow_br, q_load)
                                                
                #축열조는 계산하지 않음
                # 냉동기와 냉각탑 계산
                    temp_in_br_ch = temp_br_out_hx
                temp_out_br_ch = df['temp_out_br_ch'+str(train)][time]
                plr, temp_out_br_ch, temp_in_br_ch, q_ch_current, q_ch_available, kw_ch_current, q_cw_current, temp_cws, temp_cwr = 
                chiller(train, temp_in_br_ch, temp_out_br_ch, flow_br[train])
                
                kwh_br_pump[train] += kw_br_pump / 60
                kwh_chw_pump[train] += kw_chw_pump / 60
                rth_hx[train] += q_hx[train] / 3024 / 60
                kwh_cw_pump[train] += kw_cw_pump[train] / 60kwh_ctfan[train] += kw_ctfan[train] / 60
                kwh_ch_current[train] += kw_ch_current[train] / 60
                rth_ch_current[train] += q_ch_current[train] / 3024 / 60
                rth_cw_current[train] += q_cw_current[train] / 3024 / 60
                                                
        elif mode == 0.8: # 축열조 단독 운전
            kwh_br_pump = [0] * 5
            kwh_chw_pump = [0] * 5
            for t_min in range(1, 16):
                open_icestor_mixvalve = 100
                flow_br = br_flow(open_icestor_mixvalve)
                kw_br_pump = kw_br_pump(open_icestor_mixvalve)
                                
                temp_in_br_hx, temp_out_br_hx, temp_chwr, kw_chw_pump = hx(train, flow_br, q_load)
                temp_br_mix_icestor = temp_in_br_hx
                temp_in_br_icestor = temp_out_br_hx
                                                
                # 축열조 방열 계산
                temp_out_br_icestor, temp_br_mix_icestor, flow_br_thru_icestor, q_discharge_icestor, rth_discharge_icestor, discharge_percent_icestor, iceinventory, 
                temp_water_icestor, kwh_ringblower = discharge(iceinventory, discharge_percent_icestor, flow_br, temp_in_br_icestor, temp_br_mix_icestor, temp_water_icestor) :
                                
                # 냉동기와 냉각탑 계산을 하지 않음 
                
                # temp_br_mix_icestor_new가 temp_in_br_hx와 깉지 않으면 open_icestor_mixvalve를 감소시켜 다시 계산
                
                kwh_br_pump[train] += kw_br_pump / 60
                kwh_chw_pump[train] += kw_chw_pump / 60
                rth_hx[train] += q_hx[train] / 3024 / 60
                rth_discharge_icestor[train] += q_discharge_icestor[train] / 3024 / 60
                                
        elif mode == 0: # 운전 정지
            # 이전 값 참조
            
        else:
            pass
    
        

SyntaxError: unmatched ')' (2754198852.py, line 12)